# Importing and Preprocessing the Data


In [1]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [4]:
df = pd.read_csv("dataset.csv")
print(df.columns.tolist())
df.head()

['Unnamed: 0', 'SystemCodeNumber', 'Capacity', 'Occupancy', 'LastUpdatedDate', 'LastUpdatedTime', 'IsSpecialDay', 'VehicleType', 'Latitude', 'Longitude', 'TrafficConditionNearby', 'QueueLength']


,Unnamed: 0,SystemCodeNumber,Capacity,Occupancy,LastUpdatedDate,LastUpdatedTime,IsSpecialDay,VehicleType,Latitude,Longitude,TrafficConditionNearby,QueueLength
0,0,BHMBCCMKT01,577,61,04-10-2016,07:59:42,0,car,28.5,77.15,low,2
1,1,BHMBCCMKT01,577,64,04-10-2016,08:25:42,0,car,28.5,77.15,average,2
2,2,BHMBCCMKT01,577,80,04-10-2016,08:59:42,0,car,28.5,77.15,low,2
3,3,BHMBCCMKT01,577,107,04-10-2016,09:32:46,0,car,28.5,77.15,low,3
4,4,BHMBCCMKT01,577,150,04-10-2016,09:59:48,0,car,28.5,77.15,low,3


In [5]:
df["timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                  format="%d-%m-%Y %H:%M:%S")

# Confirm it worked
print(df[["LastUpdatedDate", "LastUpdatedTime", "timestamp"]].head())

  LastUpdatedDate LastUpdatedTime           timestamp
0      04-10-2016        07:59:42 2016-10-04 07:59:42
1      04-10-2016        08:25:42 2016-10-04 08:25:42
2      04-10-2016        08:59:42 2016-10-04 08:59:42
3      04-10-2016        09:32:46 2016-10-04 09:32:46
4      04-10-2016        09:59:48 2016-10-04 09:59:48


In [6]:
print(df.columns.tolist())


['Unnamed: 0', 'SystemCodeNumber', 'Capacity', 'Occupancy', 'LastUpdatedDate', 'LastUpdatedTime', 'IsSpecialDay', 'VehicleType', 'Latitude', 'Longitude', 'TrafficConditionNearby', 'QueueLength', 'timestamp']


In [7]:
df = df.rename(columns={
    "SystemCodeNumber": "lot_id",
    "Capacity": "capacity",
    "Occupancy": "occupancy",
    "QueueLength": "queue_len",
    "IsSpecialDay": "is_special_day",
    "VehicleType": "vehicle_type",
    "TrafficConditionNearby": "traffic_level",
    "Latitude": "lat",
    "Longitude": "lon"
})


In [8]:
# Calculate occupancy ratio
df["occ_ratio"] = df["occupancy"] / df["capacity"]

# Map traffic level to numeric
traffic_map = {"low": 0.3, "average": 0.6, "high": 1.0}
df["traffic_score"] = df["traffic_level"].map(traffic_map)

# Map vehicle type to numeric weight
vehicle_map = {"car": 1.0, "bike": 0.6, "truck": 1.5}
df["vehicle_weight"] = df["vehicle_type"].map(vehicle_map)


# MODEL 1 – Baseline Linear Pricing

In [9]:
def baseline_price(prev_price, occ_ratio, alpha=2.0):
    return prev_price + alpha * occ_ratio

# Filter data for a specific parking lot
lot_df = df[df["lot_id"] == "BHMBCCMKT01"].sort_values("timestamp")

# Initialize
price = 10.0
prices = []

# Generate price for each timestamp
for occ in lot_df["occ_ratio"]:
    price = baseline_price(price, occ)
    price = max(5.0, min(20.0, price))  # Keep price between $5 and $20
    prices.append(price)

lot_df["baseline_price"] = prices

# Show first few rows
lot_df[["timestamp", "occ_ratio", "baseline_price"]].head()



,timestamp,occ_ratio,baseline_price
0,2016-10-04 07:59:42,0.105719,10.211438
1,2016-10-04 08:25:42,0.110919,10.433276
2,2016-10-04 08:59:42,0.138648,10.710572
3,2016-10-04 09:32:46,0.185442,11.081456
4,2016-10-04 09:59:48,0.259965,11.601386


In [10]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

p = figure(title="Model 1: Baseline Pricing for Lot BHMBCCMKT01",
           x_axis_type="datetime", x_axis_label='Time', y_axis_label='Price')

p.line(lot_df["timestamp"], lot_df["baseline_price"], line_width=2, color="green")
show(p)


# MODEL 2: Demand-Based


In [11]:
def demand_score(row, weights):
    return (
        weights["occ"] * row["occ_ratio"] +
        weights["queue"] * row["queue_len"] +
        weights["traffic"] * row["traffic_score"] +
        weights["special"] * row["is_special_day"] +
        weights["vehicle"] * row["vehicle_weight"]
    )


In [12]:
def demand_price(base_price, demand, lam=0.25):
    price = base_price * (1 + lam * demand)
    return max(base_price * 0.5, min(base_price * 2.0, price))  # Clamp between 5 and 20


In [13]:
weights = {
    "occ": 0.5,
    "queue": 0.3,
    "traffic": 0.2,
    "special": 1.0,
    "vehicle": 0.2
}

lot_df = df[df["lot_id"] == "BHMBCCMKT01"].sort_values("timestamp")

# Initialize
base_price = 10.0
prices = []

# Apply demand pricing
for _, row in lot_df.iterrows():
    d = demand_score(row, weights)
    p = demand_price(base_price, d)
    prices.append(p)

lot_df["demand_price"] = prices
lot_df[["timestamp", "occ_ratio", "demand_price"]].head()


,timestamp,occ_ratio,demand_price
0,2016-10-04 07:59:42,0.105719,12.282149
1,2016-10-04 08:25:42,0.110919,12.438648
2,2016-10-04 08:59:42,0.138648,12.323310
3,2016-10-04 09:32:46,0.185442,13.131802
4,2016-10-04 09:59:48,0.259965,13.224957


In [14]:
p2 = figure(title="Model 2: Demand-Based Pricing – Lot BHMBCCMKT01",
           x_axis_type="datetime", x_axis_label='Time', y_axis_label='Price')

p2.line(lot_df["timestamp"], lot_df["demand_price"], line_width=2, color="blue")
show(p2)


In [15]:
lot_df = df[df["lot_id"] == "BHMBCCMKT01"].sort_values("timestamp").copy()

# --- Model 1: Baseline Linear Pricing ---
def baseline_price(prev_price, occ_ratio, alpha=2.0):
    return prev_price + alpha * occ_ratio

price = 10.0
baseline_prices = []
for occ in lot_df["occ_ratio"]:
    price = baseline_price(price, occ)
    price = max(5.0, min(20.0, price))
    baseline_prices.append(price)

lot_df["baseline_price"] = baseline_prices

# --- Model 2: Demand-Based Pricing ---
def demand_score(row, weights):
    return (
        weights["occ"] * row["occ_ratio"] +
        weights["queue"] * row["queue_len"] +
        weights["traffic"] * row["traffic_score"] +
        weights["special"] * row["is_special_day"] +
        weights["vehicle"] * row["vehicle_weight"]
    )

def demand_price(base_price, demand, lam=0.25):
    price = base_price * (1 + lam * demand)
    return max(base_price * 0.5, min(base_price * 2.0, price))

weights = {
    "occ": 0.5,
    "queue": 0.3,
    "traffic": 0.2,
    "special": 1.0,
    "vehicle": 0.2
}

base_price = 10.0
demand_prices = []
for _, row in lot_df.iterrows():
    d = demand_score(row, weights)
    p = demand_price(base_price, d)
    demand_prices.append(p)

lot_df["demand_price"] = demand_prices



In [16]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

p_compare = figure(title="Model 1 vs Model 2 – Lot BHMBCCMKT01",
                   x_axis_type="datetime", x_axis_label='Time', y_axis_label='Price')

p_compare.line(lot_df["timestamp"], lot_df["baseline_price"], line_width=2, color="green", legend_label="Model 1: Baseline")
p_compare.line(lot_df["timestamp"], lot_df["demand_price"], line_width=2, color="blue", legend_label="Model 2: Demand-Based")
p_compare.legend.location = "top_left"

show(p_compare)


# Model 3: Competitive Pricing

In [17]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c  # distance in km


In [18]:
# Get the latest timestamp for each lot
latest_timestamps = df.groupby("lot_id")["timestamp"].max().reset_index()

# Merge to get the latest row per lot
latest_df = df.merge(latest_timestamps, on=["lot_id", "timestamp"], how="inner")

# Add occupancy ratio again if needed
latest_df["occ_ratio"] = latest_df["occupancy"] / latest_df["capacity"]


In [19]:
def competitive_price(my_row, all_lots, base_price=10.0, lam=0.25, dist_threshold=1.5):
    # Compute price from demand
    demand = demand_score(my_row, weights)
    own_price = demand_price(base_price, demand, lam)

    nearby_prices = []
    for _, other_row in all_lots.iterrows():
        if other_row["lot_id"] == my_row["lot_id"]:
            continue
        distance = haversine(my_row["lat"], my_row["lon"], other_row["lat"], other_row["lon"])
        if distance <= dist_threshold:
            competitor_demand = demand_score(other_row, weights)
            competitor_price = demand_price(base_price, competitor_demand, lam)
            nearby_prices.append(competitor_price)

    # Adjust price based on nearby competitors
    if len(nearby_prices) > 0:
        avg_nearby_price = np.mean(nearby_prices)
        if my_row["occ_ratio"] > 0.9 and own_price > avg_nearby_price:
            # Overburdened and expensive → drop price
            return max(own_price - 2.0, base_price * 0.5)
        elif own_price < avg_nearby_price:
            # Others are more expensive → increase price
            return min(own_price + 2.0, base_price * 2)

    return own_price  # No nearby competitors or no adjustment needed


In [20]:
competitive_prices = []

for _, row in latest_df.iterrows():
    price = competitive_price(row, latest_df)
    competitive_prices.append(price)

latest_df["competitive_price"] = competitive_prices


In [21]:
# Define demand function again (in case it's not defined)
def demand_score(row, weights):
    return (
        weights["occ"] * row["occ_ratio"] +
        weights["queue"] * row["queue_len"] +
        weights["traffic"] * row["traffic_score"] +
        weights["special"] * row["is_special_day"] +
        weights["vehicle"] * row["vehicle_weight"]
    )

def demand_price(base_price, demand, lam=0.25):
    price = base_price * (1 + lam * demand)
    return max(base_price * 0.5, min(base_price * 2.0, price))

# Weights (same as before)
weights = {
    "occ": 0.5,
    "queue": 0.3,
    "traffic": 0.2,
    "special": 1.0,
    "vehicle": 0.2
}

# Base price
base_price = 10.0

# Calculate demand-based prices
demand_prices = []
for _, row in latest_df.iterrows():
    d = demand_score(row, weights)
    p = demand_price(base_price, d)
    demand_prices.append(p)

latest_df["demand_price"] = demand_prices


In [22]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

p = figure(
    title="Model 2 vs Model 3: Demand vs Competitive Pricing (Latest)",
    x_range=latest_df["lot_id"].tolist(),
    x_axis_label='Lot ID',
    y_axis_label='Price',
    width=900
)

p.vbar(
    x=latest_df["lot_id"], top=latest_df["demand_price"],
    width=0.4, color="blue", legend_label="Model 2: Demand"
)

p.vbar(
    x=latest_df["lot_id"], top=latest_df["competitive_price"],
    width=0.25, color="red", legend_label="Model 3: Competitive"
)

p.legend.location = "top_left"
show(p)


In [23]:
df["timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                  format="%d-%m-%Y %H:%M:%S")



In [24]:
df = df.rename(columns={
    "SystemCodeNumber": "lot_id",
    "Capacity": "capacity",
    "Occupancy": "occupancy",
    "QueueLength": "queue_len",
    "IsSpecialDay": "is_special_day",
    "VehicleType": "vehicle_type",
    "TrafficConditionNearby": "traffic_level",
    "Latitude": "lat",
    "Longitude": "lon"
})


In [25]:
df["occ_ratio"] = df["occupancy"] / df["capacity"]

traffic_map = {"low": 0.3, "average": 0.6, "high": 1.0}
df["traffic_score"] = df["traffic_level"].map(traffic_map)

vehicle_map = {"car": 1.0, "bike": 0.6, "truck": 1.5}
df["vehicle_weight"] = df["vehicle_type"].map(vehicle_map)


In [26]:
weights = {
    "occ": 0.5,
    "queue": 0.3,
    "traffic": 0.2,
    "special": 1.0,
    "vehicle": 0.2
}

base_price = 10.0


In [27]:
print(df.columns.tolist())

['Unnamed: 0', 'lot_id', 'capacity', 'occupancy', 'LastUpdatedDate', 'LastUpdatedTime', 'is_special_day', 'vehicle_type', 'lat', 'lon', 'traffic_level', 'queue_len', 'timestamp', 'occ_ratio', 'traffic_score', 'vehicle_weight']


In [28]:
def demand_price(row):
    base = 10
    occ_ratio = row["occ_ratio"]
    queue = row["queue_len"]
    special = row["is_special_day"]
    traffic = row["traffic_score"]
    vehicle = row["vehicle_weight"]

    # Demand function
    demand = (
        0.5 * occ_ratio +
        0.3 * queue +
        0.2 * traffic +
        1.0 * special +
        0.2 * vehicle
    )

    # Price formula
    price = base * (1 + 0.25 * demand)
    return max(base * 0.5, min(base * 2, price))  # Clamp between $5 and $20


In [29]:
df["demand_price"] = df.apply(demand_price, axis=1)


In [30]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

lot_df = df[df["lot_id"] == "BHMBCCMKT01"].sort_values("timestamp")

p = figure(title="Daily Price Fluctuation (Model 2 - Demand-Based)",
           x_axis_type="datetime", x_axis_label="Time", y_axis_label="Price", width=900)

p.line(lot_df["timestamp"], lot_df["demand_price"], line_width=2, color="navy")
show(p)


In [35]:
print(df.columns.tolist())


['Unnamed: 0', 'lot_id', 'capacity', 'occupancy', 'LastUpdatedDate', 'LastUpdatedTime', 'is_special_day', 'vehicle_type', 'lat', 'lon', 'traffic_level', 'queue_len', 'timestamp', 'occ_ratio', 'traffic_score', 'vehicle_weight', 'demand_price']


In [36]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.layouts import layout

output_notebook()

# ✅ Prepare data
source = ColumnDataSource(data={
    "Index": list(range(len(df))),
    "demand_price": df["demand_price"]
})

# ✅ Create the figure
p = figure(
    title="🚗 Parking Prices Based on Demand",
    x_axis_label="Entry Index",
    y_axis_label="Price (₹)",
    width=800,
    height=400,
    background_fill_color="#f5f5f5",
    toolbar_location="above"
)

# ✅ Plotting line and points
p.line(x="Index", y="demand_price", source=source, line_width=3, color="teal", alpha=0.8, legend_label="Demand-Based Price")
p.circle(x="Index", y="demand_price", source=source, size=6, color="navy", alpha=0.5)

# ✅ Hover tooltip
hover = HoverTool(tooltips=[
    ("Index", "@Index"),
    ("Price", "@demand_price{0.00}")
])
p.add_tools(hover)

# ✅ Style the plot
p.legend.location = "top_left"
p.legend.click_policy = "hide"
p.title.text_font_size = "16pt"
p.xaxis.axis_label_text_font_size = "12pt"
p.yaxis.axis_label_text_font_size = "12pt"

# ✅ Show plot
show(layout([[p]]))


# Visualizing Daily Price Fluctuations with a Bokeh Plot

In [37]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c


In [38]:
def demand_score(row):
    return (
        0.5 * row["occ_ratio"] +
        0.3 * row["queue_len"] +
        0.2 * row["traffic_score"] +
        1.0 * row["is_special_day"] +
        0.2 * row["vehicle_weight"]
    )

def demand_price(demand, base_price=10, lam=0.25):
    price = base_price * (1 + lam * demand)
    return max(base_price * 0.5, min(base_price * 2, price))


In [39]:
def competitive_price(row, current_df, base_price=10.0, lam=0.25):
    # Own demand and price
    own_demand = demand_score(row)
    own_price = demand_price(own_demand, base_price, lam)

    # Nearby lots (within 1.5 km)
    nearby_prices = []
    for _, comp_row in current_df.iterrows():
        if comp_row["lot_id"] == row["lot_id"]:
            continue
        distance = haversine(row["lat"], row["lon"], comp_row["lat"], comp_row["lon"])
        if distance <= 1.5:
            comp_demand = demand_score(comp_row)
            comp_price = demand_price(comp_demand, base_price, lam)
            nearby_prices.append(comp_price)

    # Adjust price based on nearby competition
    if len(nearby_prices) > 0:
        avg_nearby = np.mean(nearby_prices)
        if row["occ_ratio"] > 0.9 and own_price > avg_nearby:
            return max(own_price - 2.0, base_price * 0.5)
        elif own_price < avg_nearby:
            return min(own_price + 2.0, base_price * 2.0)

    return own_price


In [40]:
df = df.sort_values(["timestamp", "lot_id"]).copy()
competitive_prices = []

for ts in df["timestamp"].unique():
    current_df = df[df["timestamp"] == ts]
    for _, row in current_df.iterrows():
        price = competitive_price(row, current_df)
        competitive_prices.append(price)

df["competitive_price"] = competitive_prices


In [41]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

lot_df = df[df["lot_id"] == "BHMBCCMKT01"].sort_values("timestamp")

p = figure(title="Model 3: Daily Competitive Pricing – Lot BHMBCCMKT01",
           x_axis_type="datetime", x_axis_label="Time", y_axis_label="Price", width=900)

p.line(lot_df["timestamp"], lot_df["competitive_price"], line_width=2, color="firebrick")
show(p)


In [42]:
def suggest_reroute(row, frame):
    if row["occ_ratio"] < 0.95:
        return "Not Needed"
    options = []
    for _, other in frame.iterrows():
        if other["lot_id"] == row["lot_id"]:
            continue
        dist = haversine(row["lat"], row["lon"], other["lat"], other["lon"])
        if dist <= 1.5 and other["occ_ratio"] < 0.7:
            options.append((other["lot_id"], dist))
    if options:
        return sorted(options, key=lambda x: x[1])[0][0]
    return "No better lot"

df["reroute_to"] = df.groupby("timestamp").apply(lambda group: group.apply(lambda row: suggest_reroute(row, group), axis=1)).reset_index(drop=True)


/tmp/ipython-input-42-3616636374.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df["reroute_to"] = df.groupby("timestamp").apply(lambda group: group.apply(lambda row: suggest_reroute(row, group), axis=1)).reset_index(drop=True)


# **Real-Time Simulation with Pathway**

In [43]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return R * 2 * np.arcsin(np.sqrt(a))


In [44]:
@pw.udf
def calculate_price(
    occ_ratio: float,
    queue_len: int,
    special: int,
    traffic_score: float,
    vehicle_weight: float,
    lat: float,
    lon: float,
    lot_id: str,
    timestamp: str
) -> float:
    base = 10.0
    demand = 0.5 * occ_ratio + 0.3 * queue_len + 0.2 * traffic_score + 1.0 * special + 0.2 * vehicle_weight
    price = base * (1 + 0.25 * demand)
    return max(base * 0.5, min(base * 2, price))


In [45]:
import pathway as pw

class ParkingSchema(pw.Schema):
    SystemCodeNumber        : str
    Capacity                : int
    Occupancy               : int
    LastUpdatedDate         : str
    LastUpdatedTime         : str
    IsSpecialDay            : int
    VehicleType             : str
    Latitude                : float
    Longitude               : float
    TrafficConditionNearby  : str
    QueueLength             : int


In [47]:
table = pw.io.csv.read(
    "dataset.csv",
    schema=ParkingSchema,
    mode="streaming",
    autocommit_duration_ms=1000
)


In [48]:
import pathway as pw

@pw.udf
def get_vehicle_weight(v):
    if v == "car":
        return 1.0
    elif v == "bike":
        return 0.6
    elif v == "truck":
        return 1.5
    else:
        return 1.0

@pw.udf
def get_traffic_score(t):
    if t == "low":
        return 0.3
    elif t == "average":
        return 0.6
    elif t == "high":
        return 1.0
    else:
        return 0.5


In [49]:
processed = table.select(
    lot_id = table.SystemCodeNumber,
    occ_ratio = table.Occupancy / table.Capacity,
    queue_len = table.QueueLength,
    special = table.IsSpecialDay,
    vehicle_weight = get_vehicle_weight(table.VehicleType),
    traffic_score = get_traffic_score(table.TrafficConditionNearby),
    lat = table.Latitude,
    lon = table.Longitude,
    timestamp = table.LastUpdatedDate + " " + table.LastUpdatedTime
)


In [50]:
final = processed.select(
    lot_id = processed.lot_id,
    timestamp = processed.timestamp,
    price = calculate_price(
        processed.occ_ratio,
        processed.queue_len,
        processed.special,
        processed.traffic_score,
        processed.vehicle_weight,
        processed.lat,
        processed.lon,
        processed.lot_id,
        processed.timestamp
    )
)


In [51]:
pw.io.csv.write(
    final,
    filename="live_prices.csv"
)


/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


In [ ]:
pw.run()


PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input           669                                     
    FileSystem…            0             0         1312    output          669          0                          
                                                                                                                   
                                                            Above you can see the latency of input and output      
                                                           operators. The latency is measured as the difference    
                                                          between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

In [54]:
!tail live_prices.csv


"BHMBCCMKT01","06-12-2016 13:29:16","17.74610051993068","1751902194866","1"
"BHMBCCMKT01","16-10-2016 08:27:13","14.753986135181975","1751902194866","1"
"BHMBCCMKT01","29-10-2016 07:59:48","12.410485268630849","1751902194866","1"
"BHMBCCMKT01","15-10-2016 08:34:10","11.536481802426344","1751902194866","1"
"BHMBCCMKT01","09-10-2016 08:30:45","15.490987868284227","1751902194866","1"
"BHMBCCMKT01","15-11-2016 15:03:48","13.268284228769499","1751902194866","1"
"BHMBCCMKT01","14-10-2016 10:01:11","13.440294627383016","1751902194866","1"
"BHMBCCMKT01","04-11-2016 13:59:31","17.387781629116116","1751902194866","1"
"BHMBCCMKT01","27-10-2016 11:32:32","15.709445407279027","1751902194866","1"
"BHMBCCMKT01","27-11-2016 12:32:25","20.0","1751902194866","1"


In [55]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Legend
from bokeh.palettes import Category10
from bokeh.models import ColumnDataSource
output_notebook()


In [56]:
# Re-read the file and manually assign correct column names
df = pd.read_csv("live_prices.csv", skiprows=1, names=["lot_id", "timestamp", "price", "row_id", "version"])

# Drop the accidental second header row
df = df[df["lot_id"] != "lot_id"]

# Reset index
df.reset_index(drop=True, inplace=True)

# Convert timestamp to datetime
df["timestamp"] = pd.to_datetime(df["timestamp"], format="%d-%m-%Y %H:%M:%S", errors="coerce")

# Convert price to float
df["price"] = df["price"].astype(float)

# Drop any rows with invalid timestamps
df = df.dropna(subset=["timestamp"])

# Final check
df.head()


,lot_id,timestamp,price,row_id,version
0,BHMBCCMKT01,2016-11-13 10:33:32,17.149653,1751902194866,1
1,BHMBCCMKT01,2016-10-08 16:30:42,13.690728,1751902194866,1
2,BHMBCCMKT01,2016-12-14 12:30:00,18.883449,1751902194866,1
3,BHMBCCMKT01,2016-11-23 10:31:36,14.766638,1751902194866,1
4,BHMBCCMKT01,2016-10-22 13:59:48,19.183882,1751902194866,1


# Visualizations (Using Bokeh)

In [57]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Legend
from bokeh.palettes import Category10

output_notebook()

# Step 1: Pick top 3 most frequent lots
top_lots = df["lot_id"].value_counts().nlargest(3).index.tolist()

# Step 2: Create a Bokeh plot
p = figure(title="Daily Price Fluctuations for Top 3 Parking Lots",
           x_axis_label='Timestamp',
           y_axis_label='Price',
           x_axis_type='datetime',
           width=900,
           height=400)

colors = Category10[3]
legend_items = []

# Step 3: Plot each lot's time series
for i, lot in enumerate(top_lots):
    lot_df = df[df["lot_id"] == lot].sort_values("timestamp")
    line = p.line(lot_df["timestamp"], lot_df["price"],
                  line_width=2, color=colors[i], legend_label=lot)

# Step 4: Final touches
p.legend.location = "top_left"
p.legend.click_policy = "hide"

# Step 5: Show the plot
show(p)
